In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

In [ ]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tools=[wiki_tool]

In [ ]:
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages: Annotated[list, add_messages]
    

In [ ]:
from langgraph.graph import StateGraph, START, END

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = os.getenv("GROQ_API_KEY"), model="Llama-3.3-70b-Versatile")

In [ ]:
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
def chatbot(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
user_input = "Hi there!, My name is Anand!"

events=graph.stream(
    {"messages": [("user", user_input)]}, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
user_input = "Do you know who is sharukh khan"

events=graph.stream(
    {"messages": [("user", user_input)]}, stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()